In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os

In [19]:
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
from tensorflow.keras.applications import Xception

In [14]:
from tensorflow.keras import Input

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# 데이터 로드 및 전처리
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [8]:
# 데이터셋 디렉토리 경로와 이미지 크기 설정
data_directory = 'path_to_your_dataset_directory'
height = 64
width = 64
batch_size = 64

In [30]:
# ImageDataGenerator 생성 및 rescale 인자 설정
datagen = ImageDataGenerator(rescale=1./255,
                             horizontal_flip= True,
                             vertical_flip= True,
                             zoom_range= 0.2,
                             rotation_range= 0.7,
                             validation_split=0.3)  # validation_split으로 train/validation 분할 비율 설정

# Train 데이터셋 생성
train_dataset = datagen.flow_from_dataframe(train_df, x_col= 'img_path', y_col= 'label',
                                           target_size= (width, height),
                                            batch_size = batch_size,
                                           class_mode = 'categorical',
                                            subset='training',
                                           shuffle = True)

val_dataset = datagen.flow_from_dataframe(train_df, x_col= 'img_path', y_col= 'label',
                                           target_size= (width, height),
                                            batch_size = batch_size,
                                           class_mode = 'categorical',
                                            subset='validation',
                                           shuffle = True)


Found 11084 validated image filenames belonging to 25 classes.
Found 4750 validated image filenames belonging to 25 classes.


In [10]:
train_image, train_label = next(train_dataset)

In [11]:
train_image.shape

(64, 64, 64, 3)

In [12]:
train_label.shape

(64, 25)

In [33]:
# 전이 학습
basemodel =Xception(weights = 'imagenet', # 어디 데이터를 학습할 것인지
                    include_top = False, # 분류 책임자
                    input_tensor = Input(shape = (64,64,3)))

83689472/83683744 [==============================] - 19s 0us/step


In [36]:
# freezing layers in the model 마지막 10개의 레이어만 훈련할 수 있도록
for layer in basemodel.layers[:-10]:
    layer.trainable = False

In [40]:
x = AveragePooling2D(pool_size = (2,2))(basemodel.output)
x = Flatten(name = 'flatten')(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(25, activation='softmax')(x)


In [41]:
model = Model(inputs = basemodel.input, outputs = output)

In [42]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [27]:
# using early stopping to exit training if validation loss is not decreasing after certain number of epochs (patience)
earlystopping = EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 10)
# save the best model with lower validation loss
checkpointer = ModelCheckpoint(filepath = 'Efficinet_weights.hdf5',verbose = 1, save_best_only = True)

In [43]:
history = model.fit_generator(train_dataset, epochs = 20, validation_data = val_dataset,
                              callbacks =[checkpointer, earlystopping])

C:\Users\YJ_PC\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
174/174 [==============================] - 129s 731ms/step - loss: 3.1126 - accuracy: 0.0854 - val_loss: 2.9001 - val_accuracy: 0.1526

Epoch 00001: val_loss improved from 3.21835 to 2.90011, saving model to Efficinet_weights.hdf5


C:\Users\YJ_PC\anaconda3\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/20
174/174 [==============================] - 127s 732ms/step - loss: 2.6281 - accuracy: 0.2127 - val_loss: 2.5453 - val_accuracy: 0.2352

Epoch 00002: val_loss improved from 2.90011 to 2.54534, saving model to Efficinet_weights.hdf5
Epoch 3/20
174/174 [==============================] - 128s 733ms/step - loss: 2.3675 - accuracy: 0.2870 - val_loss: 2.2590 - val_accuracy: 0.3225

Epoch 00003: val_loss improved from 2.54534 to 2.25900, saving model to Efficinet_weights.hdf5
Epoch 4/20
174/174 [==============================] - 131s 751ms/step - loss: 2.1951 - accuracy: 0.3456 - val_loss: 2.3321 - val_accuracy: 0.3171

Epoch 00004: val_loss did not improve from 2.25900
Epoch 5/20
174/174 [==============================] - 129s 741ms/step - loss: 2.0939 - accuracy: 0.3773 - val_loss: 2.1033 - val_accuracy: 0.3657

Epoch 00005: val_loss improved from 2.25900 to 2.10327, saving model to Efficinet_weights.hdf5
Epoch 6/20
 97/174 [===============>..............] - ETA: 47s - loss: 2.003

KeyboardInterrupt: 